# 1. Preprocessing

In [1]:
import pandas as pd
import os
from openpyxl import load_workbook

file_path = 'Crops.xlsx'

In [2]:
# load Soil_Conditions

def load_soil_conditions(file_path):
    # Load the workbook and select the 'Soil_Conditions' sheet
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook['Soil_Conditions']
    
    df = pd.DataFrame(sheet.values)
    df.columns = df.iloc[0]
    df = df[1:]
    
    df = df[['SoilCondition_ID', 'Condition', 'Category']]
    
    return df

soil_conditions_df = load_soil_conditions(file_path)
print(soil_conditions_df)

0  SoilCondition_ID            Condition              Category
1                S1                sandy               Texture
2                S2                silty               Texture
3                S3                 clay               Texture
4                S4                loamy               Texture
5                S5           loamy clay               Texture
6                S6           sandy loam               Texture
7                S7           silty loam               Texture
8                S8                  dry  WaterHoldingCapacity
9                S9                fresh  WaterHoldingCapacity
10              S10                  wet  WaterHoldingCapacity
11              S11              aerated              Aeration
12              S12              compact              Aeration
13              S13           compressed              Aeration
14              S14               acidic              pH-Value
15              S15    moderately acidic              p

In [3]:
# load Weed Management Strategies

def load_strategies(file_path):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook['Operations']
    df = pd.DataFrame(sheet.values)
    
    # Set the first row as header
    df.columns = df.iloc[0]
    df = df[1:]
    
    return df

strategies_df = load_strategies(file_path)
print(strategies_df)

0  Operation_ID                  Name               GermanName         Type
1            O1        manual removal        manuell Entfernen       manual
2            O2     stubble treatment       Stoppelbearbeitung   mechanical
3            O3       turning tillage            Bodenwendend    mechanical
4            O4       depth loosening            Tiefenlockern   mechanical
5            O5   shallow cultivation  flache Bodenbearbeitung   mechanical
6            O6          harrow tines                Striegeln   mechanical
7            O7            cultivator                   Hacken   mechanical
8            O8         stale seedbed        falsches Saatbeet   mechanical
9            O9       thermal control     thermische Verfahren      thermal
10          O10              chalking                   Kalken   biological
11          O11         crop rotation              Fruchtfolge  cultivation
12          O12            cover crop          Zwischenfrüchte  cultivation
13          

In [4]:
# load plant data from sheets Crops and Weeds

def load_plants(file_path):
    workbook = load_workbook(filename=file_path, data_only=True)
    plant_data_df = pd.DataFrame()
    
    sheets = ['Crops', 'Weeds']
    for sheet_name in sheets:
        sheet = workbook[sheet_name]
        df = pd.DataFrame(sheet.values)
        
        # Set the first row as header
        df.columns = df.iloc[0]
        df = df[1:]
        df = df[['Plant_ID', 'ScientificName', 'CommonName', 'GermanName', 'Family', 'Genus']]
        
        plant_data_df = pd.concat([plant_data_df, df], ignore_index=True)
    
    return plant_data_df

plants_df = load_plants(file_path)
print(plants_df)

0  Plant_ID            ScientificName CommonName                   GermanName  \
0        C1  Triticum aestivum spelta      Spelt                       Dinkel   
1        C2         Triticum dicoccum      Emmer                        Emmer   
2        C3       Triticum monococcum    Einkorn                      Einkorn   
3        C4           Hordeum vulgare     Barley                       Gerste   
4        C5                      None       None                        Hafer   
..      ...                       ...        ...                          ...   
83      W41          Rumex acetosella       None          Kleiner Sauerampfer   
84      W42          Sonchus arvensis       None            Acker-Gänsedistel   
85      W43      Teesdalia nudicaulis       None  Nacktstängeliger Bauernsenf   
86      W44          Veronica persica       None        Persischer Ehrenpreis   
87      W45           Veronica polita       None        Glänzender Ehrenpreis   

0      Family Genus  
0   S

In [5]:
# load Traits for Crops and Weeds
# select columns scientificName, HeightMax, GrowthPattern, LifeCycle, RootSystem, GerminationDepth, GerminationTemperature, GerminationLight, SeedBankDuration

def load_plant_trait_edges(file_path):
    workbook = load_workbook(filename=file_path, data_only=True)
    
    weeds_sheet = workbook['Weeds']
    df = pd.DataFrame(weeds_sheet.values)
    df.columns = df.iloc[0]
    df = df[1:]
    df = df[['ScientificName', 'HeightMax', 'GrowthPattern', 'LifeCycle', 'RootSystem', 'GerminationDepth', 'GerminationTemperature', 'GerminationLight', 'SeedBankDuration']]
    
    
    crops_sheet = workbook['Crops']
    df_crops = pd.DataFrame(crops_sheet.values)
    df_crops.columns = df_crops.iloc[0]
    df_crops = df_crops[1:]
    df_crops = df_crops[['ScientificName', 'HeightMax', 'RootSystem', 'GerminationTemperature', 'GerminationLight']]
    
    df = pd.concat([df, df_crops], ignore_index=True)
    df = df.dropna(subset=['ScientificName'])
    
    # bring them into a key-value format with scientificName, Trait, Value
    df = df.melt(id_vars=['ScientificName'], var_name='Trait', value_name='Value')
    
    
    return df

plant_trait_edges_df = load_plant_trait_edges(file_path)
print(plant_trait_edges_df)

               ScientificName             Trait Value
0      Alopecurus myosuroides         HeightMax    60
1           Anthemis arvensis         HeightMax    50
2            Sinapis arvensis         HeightMax    60
3              Viola arvensis         HeightMax    40
4             Cirsium arvense         HeightMax   150
..                        ...               ...   ...
395  Triticum aestivum spelta  SeedBankDuration   NaN
396         Triticum dicoccum  SeedBankDuration   NaN
397       Triticum monococcum  SeedBankDuration   NaN
398           Hordeum vulgare  SeedBankDuration   NaN
399         Triticum aestivum  SeedBankDuration   NaN

[400 rows x 3 columns]


In [6]:
# CropType (e.g. Cereal, Root), Season, Rows

def load_crop_croptrait_edges(file_path):
    workbook = load_workbook(filename=file_path, data_only=True)
    
    crops_sheet = workbook['Crops']
    df_crops = pd.DataFrame(crops_sheet.values)
    df_crops.columns = df_crops.iloc[0]
    df_crops = df_crops[1:]
    df_crops = df_crops[['ScientificName', 'CropUsage', 'RowSpacing', 'Season']]
    
    df_crops = df_crops.dropna(subset=['ScientificName'])
    
    # bring them into a key-value format
    df_crops = df_crops.melt(id_vars=['ScientificName'], var_name='CropTrait', value_name='CropTraitValue')
    
    
    return df_crops

crop_croptrait_edges_df = load_crop_croptrait_edges(file_path)
print(crop_croptrait_edges_df)

              ScientificName   CropTrait CropTraitValue
0   Triticum aestivum spelta   CropUsage        Cereals
1          Triticum dicoccum   CropUsage        Cereals
2        Triticum monococcum   CropUsage        Cereals
3            Hordeum vulgare   CropUsage        Cereals
4          Triticum aestivum   CropUsage        Cereals
5   Triticum aestivum spelta  RowSpacing           pure
6          Triticum dicoccum  RowSpacing           pure
7        Triticum monococcum  RowSpacing           pure
8            Hordeum vulgare  RowSpacing           pure
9          Triticum aestivum  RowSpacing           pure
10  Triticum aestivum spelta      Season         winter
11         Triticum dicoccum      Season         winter
12       Triticum monococcum      Season         winter
13           Hordeum vulgare      Season         spring
14         Triticum aestivum      Season         winter


In [7]:
# Edges from Plant to preferred SoilCondition

def load_soil_plant_edges(file_path):
    workbook = load_workbook(filename=file_path, data_only=True)
    soil_edges_df = pd.DataFrame()
    
    sheets = ['Crops', 'Weeds']
    for sheet_name in sheets:
        sheet = workbook[sheet_name]
        df = pd.DataFrame(sheet.values)
        
        df.columns = df.iloc[0]
        df = df[1:]
        df = df[['ScientificName', 'prefers_SoilCondition']]
        df = df.dropna(subset=['prefers_SoilCondition'])
        
        soil_edges_df = pd.concat([soil_edges_df, df], ignore_index=True)
        
    # go through each row and split the prefers_SoilCondition by comma, create a new row for each
    soil_edges_df = soil_edges_df.assign(prefers_SoilCondition=soil_edges_df['prefers_SoilCondition'].str.split(',')).explode('prefers_SoilCondition')
    soil_edges_df['prefers_SoilCondition'] = soil_edges_df['prefers_SoilCondition'].str.strip()
    
    # load soil conditions to get the mapping from Condition to SoilCondition_ID
    soil_conditions_df = load_soil_conditions(file_path)
    soil_edges_df = soil_edges_df.merge(soil_conditions_df, left_on='prefers_SoilCondition', right_on='SoilCondition_ID', how='left')
    
    return soil_edges_df[['ScientificName', 'Condition']]

soil_plant_edges_df = load_soil_plant_edges(file_path)
print(soil_plant_edges_df)

0              ScientificName        Condition
0    Triticum aestivum spelta       loamy clay
1    Triticum aestivum spelta       silty loam
2    Triticum aestivum spelta            fresh
3         Triticum monococcum            sandy
4         Triticum monococcum       sandy loam
..                        ...              ...
142    Amaranthus retroflexus       high_humus
143    Amaranthus retroflexus    high_nitrogen
144    Amaranthus retroflexus  high_phosphorus
145    Amaranthus retroflexus   high_potassium
146    Amaranthus retroflexus          aerated

[147 rows x 2 columns]


In [8]:
# Edges from Weed to indicated SoilCondition

def load_soil_weed_indicator_edges(file_path):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook['Weeds']
    df = pd.DataFrame(sheet.values)
    
    df.columns = df.iloc[0]
    df = df[1:]
    df = df[['ScientificName', 'indicates']]
    df = df.dropna(subset=['indicates'])
        
    # go through each row and split the prefers_SoilCondition by comma, create a new row for each
    df = df.assign(indicates_SoilCondition=df['indicates'].str.split(',')).explode('indicates_SoilCondition')
    df['indicates_SoilCondition'] = df['indicates_SoilCondition'].str.strip()
    
    # load soil conditions to get the mapping from Condition to SoilCondition_ID
    soil_df = load_soil_conditions(file_path)
    df = df.merge(soil_df, left_on='indicates_SoilCondition', right_on='SoilCondition_ID', how='left')
    
    return df[['ScientificName', 'Condition']]

soil_weed_indicator_edges_df = load_soil_weed_indicator_edges(file_path)
print(soil_weed_indicator_edges_df)

0     ScientificName            Condition
0   Sinapis arvensis           high_humus
1   Sinapis arvensis        high_nitrogen
2   Sinapis arvensis      high_phosphorus
3   Sinapis arvensis       high_potassium
4   Sinapis arvensis             alkaline
..               ...                  ...
90   Veronica polita           high_humus
91   Veronica polita        high_nitrogen
92   Veronica polita      high_phosphorus
93   Veronica polita       high_potassium
94   Veronica polita  moderately alkaline

[95 rows x 2 columns]


In [9]:
# Edges from Weeds to Crops

def load_weed_croptrait_edges(file_path):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook['Weeds']
    df = pd.DataFrame(sheet.values)
    
    df.columns = df.iloc[0]
    df = df[1:]
    df = df[['ScientificName', 'competes_with']]
    df = df.dropna(subset=['competes_with'])
    
    df = df.assign(competes_with=df['competes_with'].str.split(',')).explode('competes_with')
    df['competes_with'] = df['competes_with'].str.strip()
    
    # TODO: get ScientificName for individual crops
    
    return df[['ScientificName', 'competes_with']]

weed_croptrait_edges_df = load_weed_croptrait_edges(file_path)
print(weed_croptrait_edges_df)

0              ScientificName competes_with
1      Alopecurus myosuroides        winter
1      Alopecurus myosuroides       Cereals
2           Anthemis arvensis        winter
2           Anthemis arvensis       Cereals
3            Sinapis arvensis     wide_rows
4              Viola arvensis        winter
4              Viola arvensis       Cereals
5             Cirsium arvense       Cereals
5             Cirsium arvense  shallow_root
7     Persicaria lapathifolia        spring
7     Persicaria lapathifolia     wide_rows
8     Galinsoga quadriradiata        spring
9       Digitaria sanguinalis    Vegetables
9       Digitaria sanguinalis     wide_rows
10      Matricaria chamomilla        winter
10      Matricaria chamomilla       Cereals
10      Matricaria chamomilla    Zuckerrübe
10      Matricaria chamomilla     Kartoffel
11       Veronica hederifolia        winter
11       Veronica hederifolia       Cereals
12        Persicaria maculosa        spring
12        Persicaria maculosa   

In [10]:
# create a custom dataframe for CropTrait, CropTraitValue and IncompatibleStrategies with those 3 columns and 2 rows

croptrait_strategy_edges_df = pd.DataFrame({
    'CropTrait': ['RowSpacing', 'RowSpacing'],
    'CropTraitValue': ['pure', 'pure'],
    'IncompatibleStrategies': ['cultivator', 'row width']
})
print(croptrait_strategy_edges_df)

    CropTrait CropTraitValue IncompatibleStrategies
0  RowSpacing           pure             cultivator
1  RowSpacing           pure              row width


In [11]:
# Edges from Strategies to Weeds / Traits

def load_weed_strategy_edges(file_path):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook['Relations']
    
    df = pd.DataFrame(sheet.values)
    df.columns = df.iloc[0]
    df = df[1:]
    
    df = df[['Operation', 'TARGET', 'Details']]
    
    return df

weed_strategy_edges_df = load_weed_strategy_edges(file_path)
print(weed_strategy_edges_df)

0          Operation                  TARGET                     Details
1      crop rotation  Alopecurus myosuroides            weniger Getreide
2    depth loosening  Alopecurus myosuroides                        None
3    turning tillage  Alopecurus myosuroides                alle 5 Jahre
4          row width  Alopecurus myosuroides                         eng
5    turning tillage  Alopecurus myosuroides  einmalig nach neuem Befall
..               ...                     ...                         ...
117     harrow tines       Chenopodium album                        None
118       cultivator       Chenopodium album                        None
119  thermal control  Amaranthus retroflexus        bis Keimblattstadium
120    crop rotation  Amaranthus retroflexus            mehr Winterungen
121       cultivator  Amaranthus retroflexus                        None

[121 rows x 3 columns]


# 2. KG Creation

In [12]:
from neo4j import GraphDatabase
from config import NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD


def table_insertion(df, query_func):
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    with driver.session() as session:
        for row in df.to_dict('records'):
            session.execute_write(query_func, row)
        
        print("Nodes and relationships created successfully!")

    driver.close()

In [13]:
def insert_soil_data(tx, data):
    query = """
    MERGE (sc:SoilCondition {soilConditionId: $id, name: $condition})
    MERGE (sp:SoilProperty {name: $category})
    MERGE (sc)-[:isInstanceOf]->(sp)
    """
    tx.run(query, id=data['SoilCondition_ID'], condition=data['Condition'], category=data['Category'])

table_insertion(soil_conditions_df, insert_soil_data)

Nodes and relationships created successfully!


In [14]:
def insert_strategy_data(tx, data):
    query = """
    MERGE (ws:WeedingStrategy {strategyId: $id, name: $name, germanName: $german_name, type: $type})
    """
    tx.run(query, id=data['Operation_ID'], name=data['Name'], german_name=data['GermanName'], type=data['Type'])
    
table_insertion(strategies_df, insert_strategy_data)

Nodes and relationships created successfully!


In [15]:
def insert_plant_data(tx, data):
    # MERGE on the required properties
    query_base = """
    MERGE (p:Plant {plantId: $id, scientificName: $name})
    """
    
    tx.run(query_base, 
           id=data['Plant_ID'], 
           name=data['ScientificName'])
    
    # SET optional properties
    query_optional = """
    MATCH (p:Plant {plantId: $id})
    SET p.commonName = CASE WHEN $common_name IS NOT NULL THEN $common_name ELSE p.commonName END,
        p.germanName = CASE WHEN $german_name IS NOT NULL THEN $german_name ELSE p.germanName END,
        p.family = CASE WHEN $family IS NOT NULL THEN $family ELSE p.family END,
        p.genus = CASE WHEN $genus IS NOT NULL THEN $genus ELSE p.genus END
    """
    
    optional_params = {
        'id': data['Plant_ID'],
        'common_name': data.get('CommonName'),
        'german_name': data.get('GermanName'),
        'family': data.get('Family'),
        'genus': data.get('Genus')
    }

    tx.run(query_optional, **optional_params)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
with driver.session() as session:
    for row in plants_df.to_dict('records'):
        if pd.notna(row['Plant_ID']) and pd.notna(row['ScientificName']):
            session.execute_write(insert_plant_data, row)
    
    print("Nodes and relationships created successfully!")

driver.close()

Nodes and relationships created successfully!


In [16]:
def insert_plant_trait_edges(tx, data):
    query = """
    MATCH (p:Plant {scientificName: $scientific_name})
    MERGE (pt:PlantTrait {name: $trait})
    MERGE (ptv:PlantTraitValue {valueName: $trait_value})
    MERGE (p)-[:hasTrait]->(ptv)
    MERGE (ptv)-[:isInstanceOf]->(pt)
    """
    tx.run(query, 
           scientific_name=data['ScientificName'], 
           trait=data['Trait'], 
           trait_value=data['Value'])

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
with driver.session() as session:
    for row in plant_trait_edges_df.to_dict('records'):
        if pd.notna(row['Value']):
            session.execute_write(insert_plant_trait_edges, row)
    
    print("Nodes and relationships created successfully!")

driver.close()

Nodes and relationships created successfully!


In [17]:
def insert_crop_croptrait_edges(tx, data):
    query = """
    MATCH (p:Plant {scientificName: $scientific_name})
    MERGE (ct:CultivationTrait {name: $croptrait})
    MERGE (ctv:CultivationTraitValue {valueName: $croptrait_value})
    MERGE (p)-[:isCultivatedAs]->(ctv)
    MERGE (ctv)-[:isInstanceOf]->(ct)
    """
    tx.run(query, 
           scientific_name=data['ScientificName'], 
           croptrait=data['CropTrait'], 
           croptrait_value=data['CropTraitValue'])
    
table_insertion(crop_croptrait_edges_df, insert_crop_croptrait_edges)

Nodes and relationships created successfully!


In [18]:
def insert_soil_plant_edges(tx, data):
    query = """
    MATCH (p:Plant {scientificName: $scientific_name})
    MATCH (sc:SoilCondition {name: $condition})
    MERGE (p)-[:prefers]->(sc)
    """
    tx.run(query, 
           scientific_name=data['ScientificName'], 
           condition=data['Condition'])
    
table_insertion(soil_plant_edges_df, insert_soil_plant_edges)

Nodes and relationships created successfully!


In [19]:
def insert_soil_weed_indicator_edges(tx, data):
    query = """
    MATCH (p:Plant {scientificName: $scientific_name})
    MATCH (sc:SoilCondition {name: $condition})
    MERGE (p)-[:indicates]->(sc)
    """
    tx.run(query, 
           scientific_name=data['ScientificName'], 
           condition=data['Condition'])
    
table_insertion(soil_weed_indicator_edges_df, insert_soil_weed_indicator_edges)

Nodes and relationships created successfully!


In [20]:
def insert_weed_croptrait_edges(tx, data):
    query = """
    MATCH (p:Plant {scientificName: $scientific_name})
    MATCH (ctv:CultivationTraitValue {valueName: $value})
    MERGE (p)-[:competesWith]->(ctv)
    """
    tx.run(query, 
           scientific_name=data['ScientificName'], 
           value=data['competes_with'])
    
table_insertion(weed_croptrait_edges_df, insert_weed_croptrait_edges)

Nodes and relationships created successfully!


In [21]:
def insert_croptrait_strategy_edges(tx, data):
    query = """
    MATCH (ws:WeedingStrategy {name: $name})
    MATCH (ctv:CultivationTraitValue {valueName: $value})
    MERGE (ws)-[:isInapplicableTo]->(ctv)
    """
    tx.run(query, 
           name=data['IncompatibleStrategies'], 
           value=data['CropTraitValue'])
    
table_insertion(croptrait_strategy_edges_df, insert_croptrait_strategy_edges)

Nodes and relationships created successfully!


In [22]:
def insert_weed_strategy_edges(tx, data):
    query = """
    MATCH (ws:WeedingStrategy {name: $name})
    MATCH (p:Plant {scientificName: $scientific_name})
    MERGE (ws)-[r:isEffectiveAgainst]->(p)
    SET r.comment = CASE WHEN $details IS NOT NULL THEN $details ELSE r.comment END
    """
    tx.run(query, 
           name=data['Operation'], 
           scientific_name=data['TARGET'],
           details=data['Details'])
     
table_insertion(weed_strategy_edges_df, insert_weed_strategy_edges)

Nodes and relationships created successfully!
